# Problema de minimización de presupuesto (selección de un plan) — Formulación + solución con PuLP

**Objetivo:** Seleccionar **exactamente un** plan (bundle) de servidor que cumpla requisitos mínimos de recursos y **minimizar el costo** 



## Conjunto

$$
I=\{1,2,3\}
$$

Interpretación:
- $i=1$: Plan A (USD 5/mes)
- $i=2$: Plan B (USD 10/mes)
- $i=3$: Plan C (USD 20/mes)


## Parámetros

Se definen los parámetros del modelo para cada plan $i \in I$.

### Costos (USD/mes)

$$
c_i =
\begin{cases}
5 & \text{si } i=1 \\
10 & \text{si } i=2 \\
20 & \text{si } i=3
\end{cases}
$$

### Memoria RAM (GB)

$$
r_i =
\begin{cases}
1 & \text{si } i=1 \\
2 & \text{si } i=2 \\
4 & \text{si } i=3
\end{cases}
$$

### CPU (vCPU)

$$
u_i = 2 \quad \forall i \in I
$$

### Almacenamiento SSD (GB)

$$
s_i =
\begin{cases}
40 & \text{si } i=1 \\
60 & \text{si } i=2 \\
80 & \text{si } i=3
\end{cases}
$$

---

## Requerimientos mínimos

$$
R_{\min} = 2 \quad \text{(GB RAM)}, \qquad
U_{\min} = 2 \quad \text{(vCPU)}, \qquad
S_{\min} = 60 \quad \text{(GB SSD)}
$$


## Variables de decisión

Para cada $i \in I$:

$$
y_i=
\begin{cases}
1 & \text{si se elige el plan } i\\
0 & \text{si no se elige}
\end{cases}
$$

Naturaleza:

$$
y_i \in \{0,1\} \quad \forall i \in I
$$


## Función objetivo (Sense = MIN)

Minimizar el costo **normalizado** (para que el óptimo sea $1$):

$$
\min z=\sum_{i\in I}\frac{c_i}{10}\,y_i
$$


## Restricciones

1) Elegir exactamente un plan:

$$
\sum_{i\in I} y_i = 1
$$

2) RAM mínima:

$$
\sum_{i\in I} r_i\,y_i \ge R_{\min}
$$

3) CPU mínima:

$$
\sum_{i\in I} u_i\,y_i \ge U_{\min}
$$

4) SSD mínimo:

$$
\sum_{i\in I} s_i\,y_i \ge S_{\min}
$$


## Solución con PuLP



In [2]:
# Si no tienes PuLP instalado en tu entorno local:
# !pip install pulp

import pulp as pl

# -------------------------
# Datos
# -------------------------
I = [1, 2, 3]

c = {1: 5,  2: 10, 3: 20}   # USD/mes
r = {1: 1,  2: 2,  3: 4}    # GB RAM
u = {1: 2,  2: 2,  3: 2}    # vCPU
s = {1: 40, 2: 60, 3: 80}   # GB SSD

Rmin, Umin, Smin = 2, 2, 60

# -------------------------
# Modelo (MIN)
# -------------------------
model = pl.LpProblem("Minimizacion_Presupuesto_Seleccion_Plan", pl.LpMinimize)

# Variables binarias y_i
y = pl.LpVariable.dicts("y", I, lowBound=0, upBound=1, cat=pl.LpBinary)

# Función objetivo: min z = sum (c_i/10) y_i
model += pl.lpSum((c[i] / 10.0) * y[i] for i in I), "z"

# Restricciones
model += pl.lpSum(y[i] for i in I) == 1, "Elegir_un_plan"
model += pl.lpSum(r[i] * y[i] for i in I) >= Rmin, "RAM_min"
model += pl.lpSum(u[i] * y[i] for i in I) >= Umin, "CPU_min"
model += pl.lpSum(s[i] * y[i] for i in I) >= Smin, "SSD_min"

# -------------------------
# Resolver
# -------------------------
solver = pl.PULP_CBC_CMD(msg=True)
model.solve(solver)

# -------------------------
# Resultados
# -------------------------
status = pl.LpStatus[model.status]
z_star = pl.value(model.objective)

selected = [i for i in I if pl.value(y[i]) > 0.5][0]

print("Status:", status)
print("Plan seleccionado i* =", selected)
print("y* =", {i: int(round(pl.value(y[i]))) for i in I})
print("Costo (USD/mes) =", c[selected])
print("RAM (GB) =", r[selected], "| vCPU =", u[selected], "| SSD (GB) =", s[selected])
print("Valor óptimo z* =", z_star)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mymac/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/f6/mlh3vr3j61ndkjf71k26921c0000gn/T/41f87c459d96462786649c4577244a26-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/f6/mlh3vr3j61ndkjf71k26921c0000gn/T/41f87c459d96462786649c4577244a26-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 31 RHS
At line 36 BOUNDS
At line 40 ENDATA
Problem MODEL has 4 rows, 3 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 1 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of w